In [35]:
# Simulate daily data

import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path
from random import randint

import sys
sys.path.append(str("../"))
from server.hub.mqtt import SensorReport


now = datetime.now()
now = now - timedelta(microseconds=now.microsecond)
end = now + timedelta(hours=24)

time = now
ts = []

while time < end:
    time = time + timedelta(seconds=10)
    report = SensorReport(
        timestamp=pd.Timestamp(time),
        temperature=randint(10, 25),
        humidity=randint(30, 80),
        dew_point=randint(-10, 30),
        air_pressure=randint(1000, 1200),
        gas_resistance=randint(500, 2000),
    )

    ts.append(report)

df = pd.DataFrame(ts)
df.set_index("timestamp", inplace=True)


In [36]:

import lzma
import pickle

data_dir = Path("scratch/data")
data_dir.mkdir(parents=True, exist_ok=True)
sample_archive = data_dir / "10sec.xz"

with lzma.open(str(sample_archive), "w") as f:
    pickle.dump(df, f)

print(sample_archive)

scratch/data/10sec.xz


In [46]:
from functools import wraps
from time import time

def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        start = time()
        result = f(*args, **kw)
        end = time()
        ms = (end-start) * 1000
        print(f"{ms:2.2f} ms\n")
        return result

    return wrap

@timing
def open_ts(file_path):
    pd.read_pickle(file_path)

for file in data_dir.glob("*.xz"):
    print(file.stem)
    open_ts(file)

10sec
7.72 ms

30sec
2.81 ms

1sec
59.99 ms

minutely
2.16 ms

15sec
9.97 ms

hourly
0.48 ms

